In [46]:
%load_ext autoreload
%autoreload 2

import torch
from utility.attentionAutoEncoder import AttentionAutoEncoder, concate_data, AttentionDataset, get_return_from_batch
from torch.utils.data import DataLoader

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
model = AttentionAutoEncoder(3, 5, 2)

In [48]:
input_data = torch.randn(3, 5)
input_data

tensor([[-0.9263,  0.9470, -0.1670,  0.6672,  1.5584],
        [ 0.9220,  1.7494, -0.2369, -0.3122, -0.7025],
        [ 1.1051, -0.6438, -0.3345,  0.8968,  0.9512]])

In [49]:
model(input_data)

tensor([[ 0.1883,  0.3997,  0.6194,  0.1787, -0.1697]], grad_fn=<MmBackward0>)

In [50]:
import pandas as pd
from utility.data_processing import set_time_data

In [51]:
dfs = dict()
for ticker in ['btc', 'eth', 'doge', 'xrp', 'ada']:
  currency_metrics = pd.read_csv(f'../data/{ticker}_metrics_5_years.csv')
  set_currency_metrics = set_time_data(currency_metrics)
  dfs[ticker] = set_currency_metrics

In [52]:
res, test_res = concate_data(dfs, day_split='2023-12-31')

In [53]:
res.shape

torch.Size([3, 5, 1458])

In [54]:
X_t = res[:, :, 0]

In [55]:
model(X_t)

tensor([[ 0.0046,  0.0098,  0.0152,  0.0044, -0.0042]], grad_fn=<MmBackward0>)

In [56]:
X_t.shape[-1]

5

In [63]:
# Create DataSet
training_dataset = AttentionDataset(res)
test_dataset = AttentionDataset(test_res)

# Create DataLoader
training_dataloader = DataLoader(training_dataset, batch_size=50, shuffle=False, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

# Iterate through DataLoader
for batch in training_dataloader:
    print(batch.shape)  # Output: torch.Size([10, 5])
    break  # To print only one batch

torch.Size([50, 3, 5])


In [58]:
# get_return_from_batch(batch)

In [59]:
# model(batch)

In [60]:
import torch.nn as nn
import torch.optim as optim

In [61]:
criterion = nn.MSELoss()  # Mean Squared Error for reconstruction
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [62]:
num_epochs = 50

for epoch in range(num_epochs):
    total_loss = 0
    for X_t in training_dataloader:
        R_T = get_return_from_batch(X_t)
        # Forward pass
        R_T_hat = model(X_t)
        loss = criterion(R_T_hat, R_T)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(training_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/50], Loss: 0.1256
Epoch [2/50], Loss: 0.0629
Epoch [3/50], Loss: 0.0331
Epoch [4/50], Loss: 0.0190
Epoch [5/50], Loss: 0.0126
Epoch [6/50], Loss: 0.0098
Epoch [7/50], Loss: 0.0084
Epoch [8/50], Loss: 0.0077
Epoch [9/50], Loss: 0.0072
Epoch [10/50], Loss: 0.0069
Epoch [11/50], Loss: 0.0065
Epoch [12/50], Loss: 0.0062
Epoch [13/50], Loss: 0.0059
Epoch [14/50], Loss: 0.0057
Epoch [15/50], Loss: 0.0054
Epoch [16/50], Loss: 0.0052
Epoch [17/50], Loss: 0.0050
Epoch [18/50], Loss: 0.0048
Epoch [19/50], Loss: 0.0047
Epoch [20/50], Loss: 0.0045
Epoch [21/50], Loss: 0.0044
Epoch [22/50], Loss: 0.0042
Epoch [23/50], Loss: 0.0041
Epoch [24/50], Loss: 0.0040
Epoch [25/50], Loss: 0.0039
Epoch [26/50], Loss: 0.0038
Epoch [27/50], Loss: 0.0037
Epoch [28/50], Loss: 0.0036
Epoch [29/50], Loss: 0.0036
Epoch [30/50], Loss: 0.0035
Epoch [31/50], Loss: 0.0034
Epoch [32/50], Loss: 0.0034
Epoch [33/50], Loss: 0.0033
Epoch [34/50], Loss: 0.0033
Epoch [35/50], Loss: 0.0032
Epoch [36/50], Loss: 0.0032
E

In [64]:
threshold = avg_loss * 1.5  # Set an anomaly threshold
for X_t in test_dataloader:
    R_T = get_return_from_batch(X_t)
    R_T_hat = model(X_t)

    loss = torch.mean((R_T - R_T_hat) ** 2)  # Pixel-wise error
    break
    anomalies = loss > threshold  # Detect anomalies
    print("Anomalies detected:", anomalies.nonzero(as_tuple=True)[0].tolist())

IndexError: Dimension out of range (expected to be in range of [-3, 2], but got 3)

In [68]:
(R_T - R_T_hat)

tensor([[[0.0425, 0.0296, 0.0298, 0.0262, 0.0502]]], grad_fn=<SubBackward0>)